In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../src')

In [3]:
from pathlib import Path
import unidecode
import pandas as pd

import numpy as np
import re
from tqdm.notebook import tqdm
tqdm.pandas()

from utils import BelgeSayisiExtractor, MaddeSayisiExtractor, MevzuatNoExtractor
from eval_utils import filter_dataframe, show_false_predictions, calculate_accuracy
import constants

In [4]:
pd.options.display.max_columns = 20
pd.options.display.max_colwidth = 250

In [5]:
df = pd.read_csv("../data/kanunum-nlp-doc-analysis-dataset.csv")

In [6]:
df.shape

(4142, 16)

### Alanlar
- **id**                    : İçerik kimliği
- **kategori**              : İçerik Türü/Kategorisi
- **baslik**                : İçerik Başlığı
- **rega_no**               : İçeriğin yayınlandığı Resmi Gazete Sayısı
- **mukerrer_no**           : İçeriğin yayınlandığı Resmi Gazete Mükerrer Numarası (0 ise mükerrer olmadığını belirtir)
- **rega_tarihi**           : İçeriğin yayınlandığı Resmi Gazete Tarihi (`date:yyyy-mm-dd` formatında)
- **kurum**                 : İçeriği(Genelge, Cumhurbaşkanlığı(CB) Kararnamesi, Tüzük, Yönetmelik, Tebliğ) çıkaran kurum adı
- **mevzuat_no**            : İçeriğin ilgili numarası (Detaylar için ilgili kategoriye bakınız) 
- **belge_sayi**            : Genelge üzerindeki Sayı ifadesinin değeri (sadece işlenebilenler)
- **mevzuat_tarihi**        : İçeriğin ilgili Tarihi (`date:yyyy-mm-dd` formatında)
- **donem**                 : Komisyon Raporunun dönemini (örn: "27. Dönem")
- **sira_no**               : Komisyon Raporunun Sıra Sayısı (örn: "230", "42 ek 1" vb.)
- **madde_sayisi**          : İçeriğin(Kanun, KHK, CB Kararnamesi, Tüzük, Yönetmelik) bölümlerini (Madde sayısını ve işlenemeyen hükümleri/maddeleri, _madde metinlerinin_ içindeki "Madde <sayi> - ..." ifadeleri hariç )
- **data_text**             : İçerik metni (Büyük/Küçük Harf, Noktalama ve "\n" içermektedir)
- **url**                   : İçerik metninin esas alındığı websayfasının linki (kaynak linkler güncel olmayabilir)
- **kanunum_url**           : İçeriğin Kanunum linki

**İçerik Başlıkları**, Kanun, Resmi Gazete, Kanun Hükmünde Kararname, Cumhurbaşkanlığı Kararnamesi, Tebliğ, Yönetmelik ve Tebliğ için içeriğin adı olan başlık kullanılırken Komisyon Raporu, Genelge ve Özelgelerde tanımlayıcı olmak üzere işelenerek üretilmiş bir başlık tercih edilmiştir.

**Kurum Adı**, Kanunum veritabanına işlenmiş hali ile kullanılmaktadır. Örneğin; Üniversite Yönetmeliklerinde kurum adı ayrı ayrı üniversite adı kullanmak yerine tek bir ad ("Üniversite") olarak işlenmiştir. Kurum tespiti yapılırken Bakanlık isimlerinin değişmiş olabileceği ayrıca dikkate alınmalıdır.

## Kategoriler
 Kanun, Resmi Gazete, Kanun Hükmünde Kararname, Cumhurbaşkanlığı Kararnamesi, Genelge, Tebliğ, Yönetmelik, Komisyon Raporu, Tüzük ve Özelge olmak üzere toplam 10 kategori bulunmaktadır.

Tüm içerikler için `kategori`, `baslik`, `data_text`, `url`, `kanunum_url` alanları sunulmuş olup `url` alanı içerik metninin esas alındığı sayfa linkini belirtmektedir. Diğer dikkate alınacak alanlar aşağıdaki gibi olup detaylar alt başlıklarda verilmiştir.


| Kategori                                                               | İçerik Sayısı     | İlgili alanlar (Üst Veriler)                                                                                              |
| :-------                                                               | :------------     | :---------------------------                                                                                              |
| [Kanun](#kategori-kanun)                                               | 514               | kategori, baslik, rega_no, mukerrer_no, rega_tarihi, mevzuat_no, mevzuat_tarihi, madde_sayisi, data_text, url             |
| [Kanun Hükmünde Kararname](#kategori-kanun-hükmünde-kararname-khk)     | 91                | kategori, baslik, rega_no, mukerrer_no, rega_tarihi, mevzuat_no, mevzuat_tarihi, madde_sayisi, data_text, url             |
| [Resmi Gazete](#kategori-resmi-gazete)                                 | 546               | kategori, baslik, rega_no, mukerrer_no, rega_tarihi, data_text, url                                                       |
| [Komisyon Raporu](#kategori-komisyon-raporu)                           | 500               | kategori, baslik, donem, sira_no, data_text, url                                                                          |
| [Genelge](#kategori-genelge)                                           | 385               | kategori, baslik, kurum, mevzuat_no, belge_sayi, mevzuat_tarihi, data_text, url                                           |
| [Cumhurbaşkanlığı Kararnamesi](#kategori-cumhurbaşkanlığı-kararnamesi) | 54                | kategori, baslik, rega_no, mukerrer_no, rega_tarihi, kurum, mevzuat_no, mevzuat_tarihi, madde_sayisi, data_text, url      |
| [Tüzük](#kategori-tüzük)                                               | 82                | kategori, baslik, rega_no, mukerrer_no, rega_tarihi, kurum, mevzuat_no, mevzuat_tarihi, madde_sayisi, data_text, url      |
| [Yönetmelik](#kategori-yönetmelik)                                     | 597               | kategori, baslik, rega_no, mukerrer_no, rega_tarihi, kurum, mevzuat_no, mevzuat_tarihi, madde_sayisi, data_text, url      |
| [Tebliğ](#kategori-tebliğ)                                             | 623               | kategori, baslik, rega_no, mukerrer_no, rega_tarihi, kurum, data_text, url                                                |
| [Özelge](#kategori-özelge)                                             | 750               | kategori, baslik, kurum, mevzuat_tarihi, data_text, url M

### BELGE SAYISI

In [7]:
belge_sayisi_extractor = BelgeSayisiExtractor()

In [8]:
belge_sayisi_extractor.extract(df)

'belge_sayi' cikariliyor... -> 'belge_sayi_pred'


  0%|          | 0/385 [00:00<?, ?it/s]

In [9]:
# Çok fazla NaN etiket var
calculate_accuracy(input_df=df,
                   feature_extractor=belge_sayisi_extractor,
                   include_na=True)

Veri Sayisi: 385
Dogru Tahmin Sayisi: 86
Yanlis Tahmin Sayisi: 299
Basari Orani: 0.2234


In [10]:
# NaN olmayan etiketlerde %100 başarı
calculate_accuracy(input_df=df,
                   feature_extractor=belge_sayisi_extractor,
                   include_na=False)

Veri Sayisi: 43
Dogru Tahmin Sayisi: 43
Yanlis Tahmin Sayisi: 0
Basari Orani: 1.0


In [11]:
show_false_predictions(input_df=df,
                       feature_extractor=belge_sayisi_extractor,
                       cols=["baslik", "url", "kategori"],
                       include_na=True,
                       first_n=5)

,belge_sayi,belge_sayi_pred,baslik,url,kategori
917,undefined,B.07.0.GEL.0.44/4412-17-047504,"GELİR İDARESİ BAŞKANLIĞI, GELİR VERGİSİ KANUNU İÇ GENELGESİ SIRA NO:1998/5",http://www.gib.gov.tr/node/89371,Genelge
918,undefined,B.07.0.GEL.0.71/7100-89 - 009461,"GELİR İDARESİ BAŞKANLIĞI, VERGİ BARIŞI KANUNU UYGULAMA İÇ GENELGESİ SERİ NO: 2003/2",http://www.gib.gov.tr/node/89248,Genelge
919,undefined,B. 13.2.SGK.0.71.09.00/53>~? 3 0 MAYİS 2Oİ2,"SOSYAL GÜVENLİK KURUMU, GENELGE 2012-20",https://kms.kaysis.gov.tr/Home/Goster/73822,Genelge
922,undefined,B.13.2.SSK.5.01.08.00/Vİİİ-031-502522 ANKARA,"SOSYAL GÜVENLİK KURUMU, YATIRIMLARIN VE İSTİHDAMIN TEŞVİKİNE İLİŞKİN 26.06.2006 TARİHLİ 2006-16 SAYILI GENELGE",https://kms.kaysis.gov.tr/Home/Goster/106310,Genelge
923,undefined,B. 13.2.SSK.5.01.08.00/Vİİİ-031-946961 ANKARA,"SOSYAL GÜVENLİK KURUMU, İHALE KONUSU İŞLERDEN KAYNAKLANAN BORÇLARIN TAKİBİ HAKKINDA 06/12/2005 TARİHLİ 2005/16-358 EK SAYILI GENELGE",https://kms.kaysis.gov.tr/Home/Goster/128284,Genelge


### MADDE SAYISI

In [12]:
madde_sayisi_extractor = MaddeSayisiExtractor()

In [13]:
madde_sayisi_extractor.extract(df)

'madde_sayisi' cikariliyor... -> 'madde_sayisi_pred'


  0%|          | 0/1338 [00:00<?, ?it/s]

In [14]:
calculate_accuracy(input_df=df,
                   feature_extractor=madde_sayisi_extractor,
                   include_na=False)

Veri Sayisi: 1338
Dogru Tahmin Sayisi: 1071
Yanlis Tahmin Sayisi: 267
Basari Orani: 0.8004


In [15]:
show_false_predictions(input_df=df,
                       feature_extractor=madde_sayisi_extractor,
                       cols=["baslik", "url", "kategori"],
                       include_na=False,
                       first_n=5)

,madde_sayisi,madde_sayisi_pred,baslik,url,kategori
6,76,88,MİLLÎ EĞİTİM BAKANLIĞININ TEŞKİLAT VE GÖREVLERİ HAKKINDA KANUN HÜKMÜNDE KARARNAME İLE BAZI KANUN VE KANUN HÜKMÜNDE KARARNAMELERDE DEĞİŞİKLİK YAPILMASINA DAİR KANUN,https://www.resmigazete.gov.tr/eskiler/2016/12/20161209-5.htm,Kanun
8,17,94,YÜKSEKÖĞRETİM KANUNU İLE BAZI KANUN VE KANUN HÜKMÜNDE KARARNAMELERDE DEĞİŞİKLİK YAPILMASINA DAİR KANUN,https://www.resmigazete.gov.tr/eskiler/2018/05/20180518-1.htm,Kanun
11,70,80,VERGİ KANUNLARI İLE BAZI KANUN VE KANUN HÜKMÜNDE KARARNAMELERDE DEĞİŞİKLİK YAPILMASINA DAİR KANUN,https://www.resmigazete.gov.tr/eskiler/2019/01/20190118-1.htm,Kanun
14,35,61,GELİR VERGİSİ KANUNU İLE BAZI KANUNLARDA DEĞİŞİKLİK YAPILMASI HAKKINDA KANUN,http://www.resmigazete.gov.tr/eskiler/2019/07/20190719M1-1.htm,Kanun
15,19,22,"ÖDEME VE MENKUL KIYMET MUTABAKAT SİSTEMLERİ, ÖDEME HİZMETLERİ VE ELEKTRONİK PARA KURULUŞLARI HAKKINDA KANUN İLE BAZI KANUNLARDA DEĞİŞİKLİK YAPILMASINA DAİR KANUN",https://www.resmigazete.gov.tr/eskiler/2019/11/20191122-2.htm,Kanun


### MEVZUAT NO

In [16]:
mevzuat_no_extractor = MevzuatNoExtractor()

In [17]:
mevzuat_no_extractor.extract(df)

'mevzuat_no' cikariliyor... -> 'mevzuat_no_pred'


  0%|          | 0/1723 [00:00<?, ?it/s]

In [18]:
calculate_accuracy(input_df=df,
                   feature_extractor=mevzuat_no_extractor,
                   include_na=False)

Veri Sayisi: 1091
Dogru Tahmin Sayisi: 1073
Yanlis Tahmin Sayisi: 18
Basari Orani: 0.9835


In [19]:
show_false_predictions(input_df=df,
                       feature_extractor=mevzuat_no_extractor,
                       cols=["baslik", "url", "kategori", "data_text"],
                       include_na=False,
                       first_n=25)

,mevzuat_no,mevzuat_no_pred,baslik,url,kategori,data_text
1015,2007/ 1,2007/1,"GELİR İDARESİ BAŞKANLIĞI, VERGİ KİMLİK NUMARASI İÇ GENELGESİ SERİ NO: 2007/ 1",http://www.gib.gov.tr/node/89380,Genelge,VERGİ KİMLİK NUMARASI İÇ GENELGESİ SERİ NO: 2007/ 1\n\nTarih 06/07/2007\n\nSayı B.07.1.GİB.0.05.70/7033-158-60338\n\nKapsam \n\nT.C.\n\nMALİYE BAKANLIĞI\n\nGelir İdaresi BaşkanlığI\n\nSayı: B.07.1.GİB.0.05.70/7033-158-60338\n\nVERGİ KİMLİK NUMARA...
1016,2006/ 1,2006/1,"GELİR İDARESİ BAŞKANLIĞI, VERGİ KİMLİK NUMARASI İÇ GENELGESİ SERİ NO: 2006/ 1",http://www.gib.gov.tr/node/89369,Genelge,VERGİ KİMLİK NUMARASI İÇ GENELGESİ SERİ NO: 2006/ 1\n\nTarih 31/10/2006\n\nSayı B.07.1.GİB.0.05.70/7033-158-84223\n\nKapsam \n\nT.C.\n\nMALİYE BAKANLIĞI\n\nGelir İdaresi Başkanlığı\n\nSayı : B.07.1.GİB.0.05.70/7033-158-84223\n\nVERGİ KİMLİK NUMAR...
1018,1999 / 5,1999/5,"GELİR İDARESİ BAŞKANLIĞI, UYGULAMA İÇ GENELGESİ SERİ NO : 1999 / 5",http://www.gib.gov.tr/node/89378,Genelge,UYGULAMA İÇ GENELGESİ SERİ NO : 1999 / 5\n\nTarih 01/12/1999\n\nSayı B.07.0.GEL.0.71-7100-55/52802\n\nKapsam \n\nT.C.\n\nMALİYE BAKANLIĞI\n\nGelirler Genel Müdürlüğü\n\nSAYI : B.07.0.GEL.0.71-7100-55/52802\n\nUYGULAMA İÇ GENELGESİ SERİ NO : 1999 ...
1105,2001/4,2001/14,"Tapu ve Kadastro Genel Müdürlüğü, 2001/14 Sayılı ""Bölge Arşivleri"" Konulu Genelge",http://www.tkgm.gov.tr/tr/icerik/200114-sayili-bolge-arsivleri-konulu-genelge,Genelge,"Tapu ve Kadastro Genel Müdürlüğü, 2001/14 Sayılı ""Bölge Arşivleri"" Konulu Genelge \n\nT.C. BAŞBAKANLIK \n\nTAPU VE KADASTRO GENEL MÜDÜRLÜĞÜ \n\nTapu Arşiv Dairesi Başkanlığı \n\nSayı : B021TKG0150006-4109 Ankara \n\nKonu: \n\nGenelge \n\nTAPU VE ..."
1125,1667,2008/17,"Tapu ve Kadastro Genel Müdürlüğü, 2008/17 Sayılı ""Vakıflar Yönetmeliği Hk. (2008/9 sayılı Genelgeye ektir)"" Konulu Genelge",http://www.tkgm.gov.tr/tr/icerik/200817-sayili-vakiflar-yonetmeligi-hk-20089-sayili-genelgeye-ektir-konulu-genelge,Genelge,"Tapu ve Kadastro Genel Müdürlüğü, 2008/17 Sayılı ""Vakıflar Yönetmeliği Hk. (2008/9 sayılı Genelgeye ektir)"" Konulu Genelge \n\nT.C.BAYINDIRLIK VE İSKAN BAKANLIĞI \n\nTapu ve Kadastro Genel Müdürlüğü Tasarruf İşlemleri Dairesi Başkanlığı \n\nSayı ..."
1126,1678,2009/7,"Tapu ve Kadastro Genel Müdürlüğü,2009/7 Sayılı ""Kadastro Çalışmalarında Tespit Harici Bırakılan Yerlerde Uygulama"" Konulu Genelge",http://www.tkgm.gov.tr/tr/icerik/20097-sayili-kadastro-calismalarinda-tespit-harici-birakilan-yerlerde-uygulama-konulu-genelge,Genelge,"Tapu ve Kadastro Genel Müdürlüğü,2009/7 Sayılı ""Kadastro Çalışmalarında Tespit Harici Bırakılan Yerlerde Uygulama"" Konulu Genelge \n\n\n\n\n\nT.C.BAYINDIRLIK VE İSKAN BAKANLIĞI \n\nTapu ve Kadastro Genel Müdürlüğü Tasarruf İşlemleri Dairesi Başka..."
1139,1046330,2017/1,"SOSYAL GÜVENLİK KURUMU, 2017/1 SAYILI GENELGE",https://kms.kaysis.gov.tr/Home/Goster/105981,Genelge,T.C.\nSOSYAL GÜVENLİK KURUMU BAŞKANLIĞI \n\nStrateji Geliştirme Başkanlığı\n\ni rSayı\nU\n\n32945953-010.06.01-E. 1046330 23/02/2017\nKonu-: Strateji Geliştirme Başkanlığı Daire Başkanlıktan \n\nve Alt Birimlerin Görev Tanımlan\ni\n\nİÇ GENELGE \...
1140,2017/07,2017/7,"SOSYAL GÜVENLİK KURUMU, 2017/7 SAYILI GENELGE SİGORTA PRİMLERİNİN ERTELENMESİ",https://kms.kaysis.gov.tr/Home/Goster/96320,Genelge,"T.C.\nSOSYAL GÜVENLİK KURUMU BAŞKANLIĞI \n\nSigorta Primleri Genel Müdürlüğü\n\nSayı : 89075644-202.99-E.695085 07/02/2017\nKonu : Sigorta primlerinin ertelenmesi\n\nGENELGE\n2017-7\n\n18/1/2017 tarihli, 6770 sayılı Türkiye Cumhuriyeti Emekli San..."
1152,2015/28,2014/28,"SOSYAL GÜVENLİK KURUMU, 2014/28 SAYILI GENELGE",https://kms.kaysis.gov.tr/Home/Goster/69818,Genelge,"TC\nsosyal güvenlik kurumu başkanliği\n\nSigorta Primleri Genel Müdürlüğü\n\nSayı : 82200845/01006/6654040 30/12/2015\nKonu : 2014/28 Sayılı Genelge\n\nGENELGE \n2015 / 2j3\n\n20/10/2014 tarihli ve 2014/28 sayılı Genelgenin ""Çakışan süreler içind..."
1158,2015/11,2015/h,"SOSYAL GÜVENLİK KURUMU, 2014/26 SAYILI GENELGEDE GSS SÜRELERİYLE İLİGİLİ YAPILAN DEĞİŞİKİLİKLER",htt

In [20]:
constants.COLS_MEVZUAT_NO

['Kanun',
 'Kanun Hükmünde Kararname',
 'Genelge',
 'Cumhurbaşkanlığı Kararnamesi',
 'Tüzük',
 'Yönetmelik']

In [21]:
filtered_df = filter_dataframe(input_df=df,
                               selected_categories=["Genelge"],
                               custom_filter_array=[df.url.str.contains("kaysis.gov.tr")]
                              )

calculate_accuracy(input_df=filtered_df,
                   feature_extractor=mevzuat_no_extractor,
                   include_na=True)

show_false_predictions(input_df=filtered_df,
                       feature_extractor=mevzuat_no_extractor,
                       cols=["baslik", "url", "kategori", "data_text"],
                       include_na=True,
                       first_n=20)

Veri Sayisi: 154
Dogru Tahmin Sayisi: 142
Yanlis Tahmin Sayisi: 12
Basari Orani: 0.9221


,mevzuat_no,mevzuat_no_pred,baslik,url,kategori,data_text
1139,1046330,2017/1,"SOSYAL GÜVENLİK KURUMU, 2017/1 SAYILI GENELGE",https://kms.kaysis.gov.tr/Home/Goster/105981,Genelge,T.C.\nSOSYAL GÜVENLİK KURUMU BAŞKANLIĞI \n\nStrateji Geliştirme Başkanlığı\n\ni rSayı\nU\n\n32945953-010.06.01-E. 1046330 23/02/2017\nKonu-: Strateji Geliştirme Başkanlığı Daire Başkanlıktan \n\nve Alt Birimlerin Görev Tanımlan\ni\n\nİÇ GENELGE \...
1140,2017/07,2017/7,"SOSYAL GÜVENLİK KURUMU, 2017/7 SAYILI GENELGE SİGORTA PRİMLERİNİN ERTELENMESİ",https://kms.kaysis.gov.tr/Home/Goster/96320,Genelge,"T.C.\nSOSYAL GÜVENLİK KURUMU BAŞKANLIĞI \n\nSigorta Primleri Genel Müdürlüğü\n\nSayı : 89075644-202.99-E.695085 07/02/2017\nKonu : Sigorta primlerinin ertelenmesi\n\nGENELGE\n2017-7\n\n18/1/2017 tarihli, 6770 sayılı Türkiye Cumhuriyeti Emekli San..."
1152,2015/28,2014/28,"SOSYAL GÜVENLİK KURUMU, 2014/28 SAYILI GENELGE",https://kms.kaysis.gov.tr/Home/Goster/69818,Genelge,"TC\nsosyal güvenlik kurumu başkanliği\n\nSigorta Primleri Genel Müdürlüğü\n\nSayı : 82200845/01006/6654040 30/12/2015\nKonu : 2014/28 Sayılı Genelge\n\nGENELGE \n2015 / 2j3\n\n20/10/2014 tarihli ve 2014/28 sayılı Genelgenin ""Çakışan süreler içind..."
1158,2015/11,2015/h,"SOSYAL GÜVENLİK KURUMU, 2014/26 SAYILI GENELGEDE GSS SÜRELERİYLE İLİGİLİ YAPILAN DEĞİŞİKİLİKLER",https://kms.kaysis.gov.tr/Home/Goster/57268,Genelge,T.C.\nSOSYAL GÜVENLİK KURUMU BAŞKANLIĞI \n\nSigorta Primleri Genel Müdürlüğü\n\nSayı : 63665751 - /Ü» <#./<#/2015\nKonu : 2014/26 sayılı Genelgede GSS \nsüreleriyle ilgili yapılan değişiklikler\n\nGENELGE \n2015- H\n\n6552 sayılı Kanunla 5510 say...
1160,2015/9,2015/^,"SOSYAL GÜVENLİK KURUMU, 3201 SAYILI KANUNUN UYGULANMASI HAKKINDA GENELGE",https://kms.kaysis.gov.tr/Home/Goster/68882,Genelge,"T.C.\nSOSYAL GÜVENLİK KURUMU BAŞKANLIĞI \n\nEmeklilik Hizmetleri Genel Müdürlüğü\n\nSayı : 70809318.010.06.02/ Uf ^ Ol/ti/2015\nKonu : 3201 sayılı Kanunun uygulanması\n\nGENELGE \n2015/ ^\n\n08/05/1985 tarih, 3201 sayılı “Yurtdışında Bulunan Türk..."
1162,2015/01,2015/a.,"SOSYAL GÜVENLİK KURUMU, 2014/26 SAYILI GENELGEDE YAPILAN DEĞİŞİKLİKLER",https://kms.kaysis.gov.tr/Home/Goster/57379,Genelge,"T.C.\nSOSYAL GÜVENLİK KURUMU BAŞKANLIĞI \n\nSigorta Primleri Genel Müdürlüğü\n\nSayı : 64399925/031- °5.lJLt.2015\nKonu : 2014/26 sayılı Genelgede yapılan \n\ndeğişiklikler\n\nGENELGE \n\n2015- A.\n\nBilindiği üzere, 6552 sayılı Kanunla 5510 sayı..."
1165,2014/27,2014/,"SOSYAL GÜVENLİK KURUMU, 3201 SAYILI KANUNDA YAPILAN DEĞİŞİKLİKLER HAKKINDA GENLGE",https://kms.kaysis.gov.tr/Home/Goster/68877,Genelge,T.C.\nSOSYAL GÜVENLİK KURUMU BAŞKANLIĞI\n\nEmeklilik Hizmetleri Genel Müdürlüğü\n\nSayı : 70809318.010.06.02/ J?/? Z*j l°Jl2014\nKonu : 3201 sayılı Kanunda \n\nyapılan değişiklikler\n\nGENELGE\n2014/\n\n10/9/2014 tarihli ve 6552 sayılı İş Kanunu ...
1166,2014/22,"*2014/ps,2^","SOSYAL GÜVENLİK KURUMU, AYLIK BAĞLANDIKTAN SONRA YURTDIŞINDA ÇALIŞANLAR HAKKINDA GENELGE",https://kms.kaysis.gov.tr/Home/Goster/68876,Genelge,"« T.C.\nSOSYAL GÜVENLİK KURUMU BAŞKANLIĞI \n\nEmeklilik Hizmetleri Genel Müdürlüğü\n\nSayı : 70809318.010.06.02/ £ _26| ££/.?*/2014\nKonu : Aylık bağlandıktan sonra \n\nyurtdışmda çalışanlar\n■ \\n\nGENELGE \n• 2014/ £,2^\n\n5510 sayılı Sosyal Si..."
1169,2014/11,2014///,"SOSYAL GÜVENLİK KURUMU, 2013/39 SAYILI GENELGEDE DEĞİŞİKLİK YAPILMASI HAKKINDA GENELGE",https://kms.kaysis.gov.tr/Home/Goster/68874,Genelge,T.C.\nSOSYAL GÜVENLİK KURUMU BAŞKANLIĞI \n\nEmeklilik Hizmetleri Genel Müdürlüğü\n\nSayı : 70809318.010.06.02/ 590 2% fcfylOlA\nKonu : 2013/39 sayılı Genelge\n\nGENELGE \n2014/ //\n\nSosyal güvenlik sözleşmeleri uygulamalarına ilişkin yayımlanan ...
1174,2013/35,2013/js',"SOSYAL GÜVENLİK KURUMU, ANALIK İSTİRAHAT RAPORU VE MESLEK HASTALIĞI BİLDİRİMİ HAKKINDA",https://kms.kaysis.gov.tr/Home/Goster/67782,Genelge,T.C.\nSOSYAL GÜVENLİK KURUMU BAŞKANLIĞI \n\nEmeklilik Hizmetleri Genel Müdürlüğü\n\nSayı : 25029274-1093/46/285- \nKonu : Analık İstirahat Raporları ve Meslek Llastalığı \n\n